In [ ]:
import torch 
from torch import nn 
import numpy as np 

In [ ]:
torch.manual_seed(1400)

In [ ]:
x = torch.rand(2, 3) # , 5)
x

In [ ]:
torch.sigmoid(x)

In [ ]:
torch.relu(x)

In [ ]:
x.shape

In [ ]:
x_out = torch.sum(x, dim=0) # dim 1 , 2 , ..? 
x_out

In [ ]:
x_out.shape

$$ softmax(x)_i = \frac{exp(x_i)}{\sum_{j}^{ }exp(x_j))}.$$

<br>

In [ ]:
torch.softmax(x, dim=0)

In [ ]:
l1 = nn.Linear(3, 2)

In [ ]:
result_l1 = l1(x)
result_l1

In [ ]:
result_l1.shape


# ⚠️⚠️⚠️

####   • torch.nn only supports mini-batches. The entire torch.nn package only supports inputs that are a mini-batch of samples, and not a single sample.

<br>

- pip install torchtext --user -U

In [ ]:
from torchtext.datasets import AG_NEWS

- AG is a collection of more than 1 million news articles
- Consists of class ids 1-4 where 1-World, 2-Sports, 3-Business, 4-Sci/Tech

In [ ]:
train_iter = AG_NEWS(root='${HOME}/.data', split='train')

In [ ]:
next(train_iter)

In [ ]:
from torchtext.data.utils import get_tokenizer

- Also look at nltk, spacy

In [ ]:
sample_data = ['a', 'a', 'b', 'c', 'a', 'c', 'd', 'd']
sample_counter = Counter(sample_data)

In [ ]:
sample_counter

In [ ]:
tokenizer = get_tokenizer('basic_english')
train_iter = list(AG_NEWS(split='train'))

In [ ]:
print(train_iter[0])

In [ ]:
tokenizer(train_iter[0][1])

## NLTK

- pip install nltk
- python shell for downloading data:

    \>>> import nltk 

    \>>> nltk.download()

In [ ]:
from nltk.corpus import stopwords

In [ ]:
stopwords.words('english')

In [ ]:
all_words = set()
for (label, line) in train_iter:
    all_words.update(tokenizer(line))

In [ ]:
len(all_words)

In [ ]:
sorted(list(all_words))

In [ ]:
import string

In [ ]:
string.punctuation

In [ ]:
from typing import List
class Translator:
    def __init__(self, string_data: List[str]):
        self.all_words = set()
        trans_dict = str.maketrans('', '', string.punctuation)
        for (label, line) in train_iter:
            tokenized_line = tokenizer(line)
            line = [l.translate(trans_dict) for l in tokenized_line]
            self.all_words.update(line)
        self.word_count = len(self.all_words)
        
    def translate(self, string):
        pass 

In [ ]:
t = Translator((i[1] for i in train_iter))

In [ ]:
t.word_count

In [ ]:
class CounterModel(nn.Module):
    def __init__(self, input_dim = 95810, output_dim = 4):
        super().__init__()
        self.w1 = nn.Linear(input_dim, 2000)
        self.w2 = nn.Linear(2000, output_dim)
    
    def forward(self, x):
        x = torch.relu(self.w1(x))
        x = torch.softmax(self.w2, dim=1)
        return x 

In [ ]:
model = CounterModel()